In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, monotonically_increasing_id, when, first
from pyspark.sql.window import Window
import os

# 1. Tworzymy sesję Spark
spark = SparkSession.builder.appName("FASTQ_Pivot").getOrCreate()

# 2. Ścieżka do katalogu z FASTQ
fastq_dir = "/Volumes/databrics_2/default/fastq"

# 3. Lista plików FASTQ
files = ["SRR16356246_1.fastq", "SRR16356246_2.fastq"]

for file_name in files:
    file_path = os.path.join(fastq_dir, file_name)
    
    # 4. Wczytanie pliku do DataFrame Spark
    lines_df = spark.read.text(file_path)
    
    # 5. Ćwiczenie 8.2 - Dodanie numeru linii (line_number)
    window = Window.orderBy(monotonically_increasing_id())
    lines_with_id = lines_df.withColumn("line_number", row_number().over(window) - 1)
    
    # 6. Ćwiczenie 8.3 - Określenie typu linii FASTQ (line_type)
    lines_typed = lines_with_id.withColumn(
        "line_type",
        when(col("line_number") % 4 == 0, "header")
        .when(col("line_number") % 4 == 1, "sequence")
        .when(col("line_number") % 4 == 2, "separator")
        .when(col("line_number") % 4 == 3, "quality")
    )
    
    # 7. Ćwiczenie 8.4 - Utworzenie identyfikatora odczytu (record_id)
    lines_with_record = lines_typed.withColumn(
        "record_id",
        (col("line_number") / 4).cast("integer")
    )
    
    # 8. Ćwiczenie 8.5 - Pivot: przekształcenie do formatu szerokiego
    fastq_wide = lines_with_record.groupBy("record_id").pivot("line_type").agg(
        first("value")
    )
    
    # 9. Wyświetlenie pierwszych 5 rekordów w formacie szerokim
    print(f"\nPlik: {file_name} - pierwsze 5 rekordów w formacie szerokim:")
    fastq_wide.show(5, truncate=False)
    
    # 10. Sprawdzenie schematu (typy kolumn)
    fastq_wide.printSchema()
    
    # 11. Liczba odczytów w pliku
    total_reads = fastq_wide.count()
    print(f"Liczba odczytów w pliku {file_name}: {total_reads}\n")



Plik: SRR16356246_1.fastq - pierwsze 5 rekordów w formacie szerokim:


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------+
|record_id|
+---------+
+---------+

root
 |-- record_id: integer (nullable = true)

Liczba odczytów w pliku SRR16356246_1.fastq: 0


Plik: SRR16356246_2.fastq - pierwsze 5 rekordów w formacie szerokim:
+---------+
|record_id|
+---------+
+---------+

root
 |-- record_id: integer (nullable = true)

Liczba odczytów w pliku SRR16356246_2.fastq: 0

